ディレクトリ・モデルの指定

In [1]:
# リポジトリの場所
home_dir = '/content/drive/My\ Drive'
# データを入れたディレクトリの名前
data_dir='pumpkin_640_480'
# ベースモデル: yolov4-tiny.conv.29 or yolov4.conv.137
model_type = 'yolov4-tiny.conv.29'

(初回のみ) インストール

In [ ]:
%cd {home_dir}
!git clone https://github.com/husty530/Yamashita-darknet.git

ビルド

In [ ]:
%cd {home_dir}/Yamashita-darknet
!rm darknet
!make

学習条件の設定

In [ ]:
###### config ######

classes = 1
batch_size = 64
subdivisions = 16
# 幅・高さは32の倍数でないといけない
width = 640
height = 480
channels = 3
# 学習回数
max_batches = 2000

####################

%cd {home_dir}/Yamashita-darknet/workspace
url = f'https://github.com/AlexeyAB/darknet/releases/download/yolov4/{model_type}'
cfg_template = 'template-' + model_type.replace('yolo', '').split('.')[0] + '.cfg'
!wget {url} -O ../{model_type}
!python3 ../config.py {data_dir} {classes} {cfg_template} {width} {height} {channels} {batch_size} {subdivisions} {max_batches}

(新規・データ変更時)train-val の切り分け -- YOLOフォーマット(space区切り.txtファイル)のみ対応

In [6]:
%cd {home_dir}/Yamashita-darknet/workspace/{data_dir}
percentage_valid = 20
!python3 ../../split.py {data_dir} {percentage_valid}

/content/drive/My Drive/Yamashita-darknet/workspace/pumpkin_640_480
completed dividing 121 image files into "train.txt" and "valid.txt".


学習開始

In [ ]:
%cd {home_dir}/Yamashita-darknet
!mkdir workspace/{data_dir}/backup
!./darknet detector train workspace/{data_dir}/.data workspace/{data_dir}/.cfg {model_type} -map -dont_show

学習再開

In [ ]:
%cd {home_dir}/Yamashita-darknet
!./darknet detector train workspace/{data_dir}/.data workspace/{data_dir}/.cfg workspace/{data_dir}/backup/_last.weights -map -dont_show

デモ

In [ ]:
%cd {home_dir}/Yamashita-darknet
!./darknet detector test\
  workspace/{data_dir}/.data\
  workspace/{data_dir}/.cfg\
  workspace/{data_dir}/backup/_final.weights\
  workspace/{data_dir}/dataset/002.png

評価

In [ ]:
%cd {home_dir}/Yamashita-darknet
!./darknet detector map\
  workspace/{data_dir}/.data\
  workspace/{data_dir}/.cfg\
  workspace/{data_dir}/backup/_final.weights